In [316]:
import pandas as pd
import re
import numpy as np

In [317]:
df_bc = pd.read_csv('data/comments/bitcoin.csv')
df_dj = pd.read_csv('data/comments/dj.csv')
df_ts = pd.read_csv('data/comments/tesla.csv')

In [318]:
from dateutil import parser
from datetime import date
import re

def convert_date(date_string):
    
    if 'ago' in str(date_string):
        today = date.today()
        return today.strftime("%d-%m-%y")
    
    if 'nan' in str(date_string):
        return
    
    dt = parser.parse(date_string)  
    return dt.strftime("%d-%m-%y")

def convert(df):
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    df['date'] = df['date'].apply(lambda x: convert_date(x))
    
    return df
    

In [319]:
df_bc = convert(df_bc)
df_dj = convert(df_dj)
df_ts = convert(df_ts)

C:\Users\cfo19\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1204: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [320]:
del df_ts['+/-']

In [321]:
df_bc = df_bc.dropna(subset=['comment'])
df_dj = df_dj.dropna(subset=['comment'])
df_ts = df_ts.dropna(subset=['comment'])

In [322]:
df_bcp = pd.read_excel('data/price/BTC.xlsx')
df_djp = pd.read_excel('data/price/DJI.xlsx')
df_tsp = pd.read_excel('data/price/TSLA.xlsx')

In [323]:
df_bcp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856 entries, 0 to 855
Data columns (total 5 columns):
High           856 non-null float64
Low            856 non-null float64
Upper Bound    856 non-null float64
Lower Bound    856 non-null float64
Date           856 non-null object
dtypes: float64(4), object(1)
memory usage: 33.5+ KB


In [324]:
df_bc_merged = pd.merge(df_bc,df_bcp, left_on="date", right_on="Date",how="inner")
df_dj_merged = pd.merge(df_dj,df_djp, left_on="date", right_on="Date",how="inner")
df_ts_merged = pd.merge(df_ts,df_tsp, left_on="date", right_on="Date",how="inner")

In [325]:
df_bc_merged = df_bc_merged.dropna(subset=['comment'])
df_dj_merged = df_dj_merged.dropna(subset=['comment'])
df_ts_merged = df_ts_merged.dropna(subset=['comment'])

In [326]:

def add_placeholder(row):
    regex = r"\b[0-9]{3,}\b"
    if re.findall(regex,row['comment']):
        predictions_list = re.findall(regex,row['comment'])
        
        for prediction in predictions_list:    
            predictionVal = float(prediction.replace('$', ''))
    
            if predictionVal < row['Upper Bound'] and predictionVal > row['High']:
                return row['comment'].replace(prediction,'[OPTIMISTIC VALUE]')
            elif predictionVal > row['Lower Bound'] and predictionVal < row['Low']:
                return row['comment'].replace(prediction,'[PESSIMISTIC VALUE]')
            
    return row['comment']

def placeholder(df):
    df.drop(['Date'], axis=1,inplace=True)
    df=df.set_index('date')
    df['updated_comment'] = df.apply(lambda x: add_placeholder(x), axis=1)
    
    return df

In [327]:
df_bc_merged = placeholder(df_bc_merged)
df_dj_merged = placeholder(df_dj_merged)
df_ts_merged = placeholder(df_ts_merged)

In [328]:
df_ts_merged['index'] = range(1, len(df_ts_merged) + 1)

In [329]:
df_ts_merged

,comment,High,Low,Upper Bound,Lower Bound,updated_comment,index
date,,,,,,,
03-05-19,You can tell the ratio of Bulls and Bears in t...,256.609985,243.490005,320.762481,182.617504,You can tell the ratio of Bulls and Bears in t...,1
03-05-19,"Right now, it has a total debt of 10.5 billion...",256.609985,243.490005,320.762481,182.617504,"Right now, it has a total debt of 10.5 billion...",2
03-05-19,I just don't know why it is up 10% for 2 days....,256.609985,243.490005,320.762481,182.617504,I just don't know why it is up 10% for 2 days....,3
03-05-19,Up 10% in 2 days! Great way to end the week. C...,256.609985,243.490005,320.762481,182.617504,Up 10% in 2 days! Great way to end the week. C...,4
03-05-19,TESLA = Waymo + Uber + Mercedes + GE batter/e...,256.609985,243.490005,320.762481,182.617504,TESLA = Waymo + Uber + Mercedes + GE batter/e...,5
03-05-19,235 is the perfect buy for 270 . it ain't fall...,256.609985,243.490005,320.762481,182.617504,[PESSIMISTIC VALUE] is the perfect buy for 270...,6
03-05-19,They dont vaccine their kids because they sell...,256.609985,243.490005,320.762481,182.617504,They dont vaccine their kids because they sell...,7
03-05-19,happy buying it for 300\r\n,256.609985,243.490005,320.762481,182.617504,happy buying it for [OPTIMISTIC VALUE]\r\n,8
03-05-19,haters gonna hate.\r\n,256.609985,243.490005,320.762481,182.617504,haters gonna hate.\r\n,9


In [330]:
df_old_data = df_ts_merged.loc['02-01-18':]
dates=df_old_data.index

In [331]:
df_old_data

,comment,High,Low,Upper Bound,Lower Bound,updated_comment,index
date,,,,,,,
02-01-18,Dropped Tesla bought $apple\r\n,322.109985,311.000000,402.637481,233.250000,Dropped Tesla bought $apple\r\n,4281
02-01-18,Let's be optimistic and tesla production of mo...,322.109985,311.000000,402.637481,233.250000,Let's be optimistic and tesla production of mo...,4282
27-12-17,Once there was a wise old man vho said get goo...,317.679993,310.750000,397.099991,233.062500,Once there was a wise old man vho said get goo...,4283
27-12-17,Musk you need tweet everyday to save this comp...,317.679993,310.750000,397.099991,233.062500,Musk you need tweet everyday to save this comp...,4284
26-12-17,"Stick a fork in this one, people. It's done.\r\n",323.940002,316.579987,404.925003,237.434990,"Stick a fork in this one, people. It's done.\r\n",4285
26-12-17,"Let it settle a bit, the opportunity to buy is...",323.940002,316.579987,404.925003,237.434990,"Let it settle a bit, the opportunity to buy is...",4286
20-12-17,Next support is $323.65 this will start the de...,333.100006,325.040009,416.375008,243.780007,Next support is $[PESSIMISTIC VALUE].65 this w...,4287
19-12-17,Watch support$328.45 after is bad news!\r\n,341.489990,330.299988,426.862487,247.724991,Watch support$[PESSIMISTIC VALUE].45 after is ...,4288
19-12-17,I have read the articles referencing the hype ...,341.489990,330.299988,426.862487,247.724991,I have read the articles referencing the hype ...,4289


In [332]:
for rows in df_old_data:
    df_old_data.drop(df_old_data.index[0])

In [333]:
df_bc_merged.to_excel("data/output/bitcoin.xlsx", sheet_name='Sheet_1')
df_dj_merged.to_excel("data/output/dowjones.xlsx", sheet_name='Sheet_1')
df_ts_merged.to_excel("data/output/tesla.xlsx", sheet_name='Sheet_1')

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/al-zf%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20With%20support%20established%20in%20the%205550.%20area,%20longs%20have%20targets%20of%20resistance%20in%20the%205884.%20and%206357.%20areas.%20Stop%20loss%20in%20the%205320.%20area.%20Shorts%20have%20targets%20of%20support%20in%20the%205320.%20and%205194.%20areas.%20Stop%20loss...Show%20more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/alkkj%20%20%20%20With%20support%20in%20the%205320.%20area%20established%20longs%20have%20targets%20of%20resistance%20in%20the%205550.%20and%206357.%20areas.%20Stop%20loss%20in%20the%205194,%20area.%20Shorts%20have%20targets%20of%20support%20in%20the%205194,%20and%205050.%20ar

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://tvc-invdn-com.akamaized.net/data/tvc_9ee4fe9ce07083e8b1b7fa6a52a03ceb.png%20These%20are%20some%20similarities%20to%20the%20previous%20bounce%20that%20I%20keep%20seeing.%20What%20do%20you%20guys%20think?%20Or%20is%20the%20inverse%20H&amp;S;%20still%20possible?
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/9kn26%20%20Watch%20the%203700.%20area%20for%20the%20establishment%20of%20support%20/%20resistance.%20IF%20support%20is%20made%20watch%20for%20resistance%20next%20in%20the%204270.%20area.%20IF%20resistance%20is%20found%20watch%20for%20support%20next%20in%20the%203185.%20area.%20%20%20%20GL
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\User

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/7ftf%20%20New%20downtrend%20line%20established%20could%20be%20part%20of%20a%20huge%20descending%20wedge%20pattern.%20Worth%20watching%20as%20bulls%20will%20need%20to%20break%20out%20of%20this%20huge%20triangle%20and%20or%20trend%20line%20in%20order%20to%20continue%20any%20up-move.%20GLTA
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/7ey5p%20%20BBands%20Tightening%20width%20now%20.1273%20(GDAX%20data)%20last%20times%20at%20this%20level%20&amp;%20lower...3/25/16%20%20%20.0256%20%2026%20day%20bull%20run.%205/17/15%20%20.0423%2021%20day%20bull%20run.%20.%207/29/16%20%20.0467%203%20days%20bear%20flat%2030%20days.%20.%209/1/16%20%20.0317%206%20day%20bull%20run.%20.%209/29/16%20...Show%20more
' with li

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://www.reuters.com/article/uk-hongkong-regulator-cryptocurrencies/hk-regulator-says-to-crackdown-on-crypto-exchanges-for-rule-violations-idUSKBN1FT0AK%20This%20is%20very%20interesting%20as%20a%20lot%20of%20the%20big%20exchanges%20are%20based%20in%20HK
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/6kkiq.%20.%20I%20know%20this%20is%20BTC%20just%20showing%20how%20alts%20mimic.%20I%20have%20my%20stops%202%20tiers%20the%20bulk%20aggressively%20under%20the%20rising%20trendline%20on%20my%20ETH%20trade%20just%20as%20with%20BTC%20we%20see%20the%20200%20on%20the%20hourly%20(100%20on%202%20hr%20and%2050%20on%204%20hr)%20coming%20-so%20%22%20thin%20...Show%20more
' with link or location/anchor > 255 characters since it

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://www.tradingview.com/x/I5NC7nPf/%20i%20give%20the%20green%20line%20a%20good%20chance%20to%20survive.%20SL%20moved%20to%20%5bPESSIMISTIC%20VALUE%5d.%20(faded%20blue%20is%20my%20lambo%20dream%20way%20and%20why%20i%20refused%20to%20sell%20lol)%20adding%20more%20at%20these%20levels
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/81scp%20%20chart%20with%20cloud%20superimposed.%20%20Price%20wants%20equilibrium%20with%20tenken%20%20sen%20(blue%20line%20)so%20they%20want%20to%20be%20together%20not%20apart.%20%20I’ve%20projected%20out%20%20where%20the%20tenken%20sen%20should%20be%20in%20a%20few%20days....that’s%20the%20%5bPESSIMISTIC%20VALUE%5d%20number.%20%20Also%20expecting...Show%20more
' with link or location/an

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/6ozp9%20Second%20long%20target%20was%20hit.%20Now%20having%201%20long%20from%20%5bPESSIMISTIC%20VALUE%5d%20and%202%20shorts%20from%2011150%20with%20SL%20of%2012k.%20:P%20Let's%20see%20how%20it%20goes%20now.%20Basically%20now%20risking%20only%20that%20already%20TP%20long%20profits%20with%20those%202%20shorts%20SL.%20At%20the%20same%20time%20locking%20in...Show%20more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/6nru9%20%20Feeling%20a%20bit%20gambling%20today%20and%20opened%20second%20long%20with%20SL%20same%20than%20with%20earlier%20(%5bPESSIMISTIC%20VALUE%5d).%20Risking%20only%20profits,%20still%20just%20tiny%20bit%20on%20positive%20even%20SL%20hits.%20Always%20protect%20the%20invested%20equity